# Settings

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]

In [ ]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

llm = MistralAI(temperature=0.0, model='mistral-small-latest')
Settings.llm = llm

embed_model = MistralAIEmbedding(model_name="mistral-embed")

Settings.embed_model = embed_model

In [4]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

URI = os.environ["NEO4J_URI"]
USERNAME = os.environ["NEO4J_USERNAME"]
PASSWORD = os.environ["NEO4J_PASSWORD"]

graph_store = Neo4jPropertyGraphStore(
    username=USERNAME,
    password=PASSWORD,
    url=URI,
)

# Building the graph

In [5]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('data/md/').load_data()

In [ ]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor, SimpleLLMPathExtractor

### Extract graph from documents
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=embed_model,
    kg_extractors=[
        ### Different LLM path extractors can be used to extract the graph from documents depending on the use case
        # DynamicLLMPathExtractor(
        #     llm=llm,
        # )
        SimpleLLMPathExtractor(
            llm=llm,
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)


### Create an index from the existing knowledge graph
# index = PropertyGraphIndex.from_existing(
#     property_graph_store=graph_store,
#     embed_model=embed_model,
# )

# Querying the graph

In [ ]:
def answer_user_query(query, index):
    """
    Answer a user query using the index and LLM. Translate the answer to French.
    """
    query_engine = index.as_query_engine(include_text=True)
    response = query_engine.query(query)
    response = query_engine.query("Translate in french: " + str(response))
    print(response)

In [9]:
query = "est-ce que le tour de france 2025 passe à toulouse ?"

In [11]:
answer_user_query(query, index)

Oui, le Tour de France 2025 passe à Toulouse. Il y a une étape qui commence et se termine à Toulouse le 16 juillet.
